**Modelo tipo 1 e 2**

* Tipo 1: demanda <= oferta

* Tipo 2: demanda >= oferta

* fim no cliente

In [ ]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *

In [ ]:
df_ori_dest = pd.read_csv('../dados/origem_porto.csv')
df_ori_trans = pd.read_csv('../dados/origem_transbordo.csv')
df_trans_porto = pd.read_csv('../dados/transbordo_porto.csv')

In [ ]:
df_supply = pd.read_csv('../dados/supply.csv')
df_cap_port = pd.read_csv('../dados/cap_porto.csv')
df_cap_trans = pd.read_csv('../dados/cap_transbordo.csv')
df_demand = pd.read_csv('../dados/demand.csv')

In [ ]:
qnt_orig = df_supply.shape[0]
qnt_trans = df_cap_trans.shape[0]
qnt_port = df_cap_port.shape[0]
qnt_cli = df_demand.shape[0]

In [ ]:
df_supply

In [ ]:
df_demand

In [ ]:
N = [i for i in range(qnt_orig)]
M = [i + qnt_orig + qnt_trans for i in range(qnt_port)]
K = [i + qnt_orig for i in range(qnt_trans)]
O = [i + qnt_orig + qnt_trans + qnt_port for i in range(qnt_cli)]

supply = {}
for i in N:
    supply[i] = df_supply['0'][i]

demand = {}
for i in O:
    demand[i] = df_demand['0'][i - O[0]]

ways = {}
for i in M:
    ways[i] = {k for k in K}

cap_transbordo = {}
for i in K:
    cap_transbordo[i] = df_cap_trans['0'][i - K[0]]

cap_port = {}
for i in M:
    cap_port[i] = df_cap_port['0'][i - M[0]]

cost = {}
for i in N:
    for j in M:
        cost[i, j, 0] = df_ori_dest[str(j - M[0])][i]
        for k in K:
            cost[i, j, k] = df_ori_trans[str(k - K[0])][i] + df_trans_porto[str(j - M[0])][k - K[0]]

K.append(0)

In [ ]:
oferta_total = 0
for i in supply:
    oferta_total += supply[i]
demanda_total = 0
for i in demand:
    demanda_total += demand[i]

cap_transbordo[0] = max(demanda_total, oferta_total)
print(oferta_total, demanda_total)

In [ ]:
m = gp.Model("probrema")

In [ ]:
X = {}
for i in N:
    for j in M:
        for k in K:
            if not k in ways[j]: continue
            for o in O:
                X[i, j, k, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}_{}_{}".format(i, j, k, o))

In [ ]:
m.setObjective(
    gp.quicksum((X[i, j, k, o] * cost[i, j, k]) for i in N for j in M for k in K for o in O if k in ways[j]),
    sense=gp.GRB.MINIMIZE
)

In [ ]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) == supply[i]
        )
else:
    for i in N:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for j in M for k in K for o in O if k in ways[j]) <= supply[i]
        )

In [ ]:
if oferta_total <= demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) == demand[o]
        )

In [ ]:
for j in M:
    m.addConstr(
        gp.quicksum(X[i, j, k, o] for i in N for k in K for o in O if k in ways[j]) <= cap_port[j]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, j, k, o] for i in N for j in M for o in O if k in ways[j]) <= cap_transbordo[k]
    )

In [ ]:
m.addConstrs(
    gp.quicksum(X[i, j, k, o] for i in N for j in M for k in K if k in ways[j]) >= 0 for o in O
)

In [ ]:
# Executa o modelo
m.update()
m.optimize()

In [ ]:
m.write("out.sol")